# Data loading, storage, and file formats

In [1]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas as pd
np.set_printoptions(precision=4)

In [2]:
%pwd

u'/home/jovyan/work'

## Reading and Writing Data in Text Format

In [3]:
!cat ch06/ex1.csv




9,10,11,12,foo

In [4]:
df = pd.read_csv('ch06/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
pd.read_table('ch06/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
!cat ch06/ex2.csv



9,10,11,12,foo

In [8]:
pd.read_csv('ch06/ex2.csv', header=None)
#pd.read_csv('ch06/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('ch06/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [10]:
!cat ch06/csv_mindex.csv
parsed = pd.read_csv('ch06/csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [11]:
list(open('ch06/ex3.txt'))

['            A         B         C\r\n',
 'aaa -0.264438 -1.026059 -0.619500\r\n',
 'bbb  0.927272  0.302904 -0.032399\r\n',
 'ccc -0.264273 -0.386314 -0.217601\r\n',
 'ddd -0.871858 -0.348382  1.100491\r\n']

In [14]:
result = pd.read_table('ch06/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [15]:
!cat ch06/ex4.csv
pd.read_csv('ch06/ex4.csv', skiprows=[0, 2, 3])







9,10,11,12,foo

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [16]:
!cat ch06/ex5.csv
result = pd.read_csv('ch06/ex5.csv')
result
pd.isnull(result)




three,9,10,11,12,foo

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [17]:
result = pd.read_csv('ch06/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [18]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('ch06/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### Reading text files in pieces

In [38]:
result = pd.read_csv('ch06/ex6.csv')
print(result.ix[0])
print(result.ix[1000])
print(result.ix[2000])

one       0.467976
two     -0.0386485
three    -0.295344
four      -1.82473
key              L
Name: 0, dtype: object
one       0.467976
two     -0.0386485
three    -0.295344
four      -1.82473
key              T
Name: 1000, dtype: object
one       0.467976
two     -0.0386485
three    -0.295344
four      -1.82473
key              1
Name: 2000, dtype: object


In [20]:
pd.read_csv('ch06/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [28]:
chunker = pd.read_csv('ch06/ex6.csv', chunksize=1000)
chunker

0
1
2
3
4
5
6
7
8
9


In [44]:
chunker = pd.read_csv('ch06/ex6.csv', chunksize=1000)

tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
    print(piece['key'].value_counts()['E'])
    
tot = tot.order(ascending=False)
#Series.add?

32
32
39
37
54
35
30
39
35
35


/opt/conda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:8: FutureWarning: order is deprecated, use sort_values(...)


In [23]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### Writing data out to text format

In [45]:
data = pd.read_csv('ch06/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [46]:
data.to_csv('ch06/out.csv')
!cat ch06/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [47]:
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [48]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [55]:
data.to_csv(sys.stdout, index=True, header=False)

0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [57]:
data.to_csv(sys.stdout, index=True, columns=['a', 'b', 'c'])

,a,b,c
0,1,2,3.0
1,5,6,
2,9,10,11.0
  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     foo


In [58]:
dates = pd.date_range('1/1/2000', periods=7)
ts = Series(np.arange(7), index=dates)
ts.to_csv('ch06/tseries.csv')
!cat ch06/tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


In [59]:
Series.from_csv('ch06/tseries.csv', parse_dates=True)

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
dtype: int64

### Manually working with delimited formats

In [60]:
!cat ch06/ex7.csv

In [61]:
import csv
f = open('ch06/ex7.csv')

reader = csv.reader(f)

In [62]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


In [63]:
lines = list(csv.reader(open('ch06/ex7.csv')))
header, values = lines[0], lines[1:]
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [64]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [65]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

In [66]:
%cat mydata.csv

one;two;three
1;2;3
4;5;6
7;8;9


### JSON data

In [67]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
              {"name": "Katie", "age": 33, "pet": "Cisco"}]
}
"""

In [68]:
import json
result = json.loads(obj)
result

{u'name': u'Wes',
 u'pet': None,
 u'places_lived': [u'United States', u'Spain', u'Germany'],
 u'siblings': [{u'age': 25, u'name': u'Scott', u'pet': u'Zuko'},
  {u'age': 33, u'name': u'Katie', u'pet': u'Cisco'}]}

In [69]:
asjson = json.dumps(result)

In [70]:
siblings = DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,25
1,Katie,33


### XML and HTML, Web scraping

**NB. The Yahoo! Finance API has changed and this example no longer works**

In [72]:
from lxml.html import parse
from urllib2 import urlopen

parsed = parse(urlopen('http://finance.yahoo.com/q/op?s=AAPL+Options'))

doc = parsed.getroot()

ImportError: No module named lxml.html

In [73]:
links = doc.findall('.//a')
links[15:20]

NameError: name 'doc' is not defined

In [74]:
lnk = links[28]
lnk
lnk.get('href')
lnk.text_content()

NameError: name 'links' is not defined

In [75]:
urls = [lnk.get('href') for lnk in doc.findall('.//a')]
urls[-10:]

NameError: name 'doc' is not defined

In [76]:
tables = doc.findall('.//table')
calls = tables[9]
puts = tables[13]

NameError: name 'doc' is not defined

In [77]:
rows = calls.findall('.//tr')

NameError: name 'calls' is not defined

In [78]:
def _unpack(row, kind='td'):
    elts = row.findall('.//%s' % kind)
    return [val.text_content() for val in elts]

In [ ]:
_unpack(rows[0], kind='th')
_unpack(rows[1], kind='td')

In [ ]:
from pandas.io.parsers import TextParser

def parse_options_data(table):
    rows = table.findall('.//tr')
    header = _unpack(rows[0], kind='th')
    data = [_unpack(r) for r in rows[1:]]
    return TextParser(data, names=header).get_chunk()

In [ ]:
call_data = parse_options_data(calls)
put_data = parse_options_data(puts)
call_data[:10]

#### Parsing XML with lxml.objectify

In [ ]:
%cd ch06/mta_perf/Performance_XML_Data

In [ ]:
!head -21 Performance_MNR.xml

In [ ]:
from lxml import objectify

path = 'Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

In [ ]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [ ]:
perf = DataFrame(data)
perf

In [ ]:
root

In [ ]:
root.get('href')

In [ ]:
root.text

## Binary data formats

In [79]:
cd ../..

/home


In [80]:
frame = pd.read_csv('ch06/ex1.csv')
frame
frame.to_pickle('ch06/frame_pickle')

IOError: File ch06/ex1.csv does not exist

In [ ]:
pd.read_pickle('ch06/frame_pickle')

### Using HDF5 format

In [ ]:
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

In [ ]:
store['obj1']

In [ ]:
store.close()
os.remove('mydata.h5')

### Interacting with HTML and Web APIs

In [ ]:
import requests
url = 'https://api.github.com/repos/pydata/pandas/milestones/28/labels'
resp = requests.get(url)
resp

In [ ]:
data[:5]

In [ ]:
issue_labels = DataFrame(data)
issue_labels

## Interacting with databases

In [ ]:
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()

In [ ]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

In [ ]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

In [ ]:
cursor.description

In [ ]:
DataFrame(rows, columns=zip(*cursor.description)[0])

In [ ]:
import pandas.io.sql as sql
sql.read_sql('select * from test', con)